Téléchargement base de données + préprocessing

Pour Linux

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz > /dev/null 2>&1
!tar zxvf imagenette2.tgz > /dev/null 2>&1

Pour MacOS

In [ ]:
# test if file exists
import os
if not os.path.exists('imagenette2.tgz'):
    !curl -O -# https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
if not os.path.exists('imagenette2'):
    !tar -zxvf imagenette2.tgz > /dev/null 2>&1

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
import os
from torch.utils.data import Dataset

means, stds = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ]
)


def get_imagenette2_loaders(root_path="./imagenette2", **kwargs):

    trainset = torchvision.datasets.ImageFolder(
        os.path.join(root_path, "train"), transform=train_transform
    )
    trainloader = torch.utils.data.DataLoader(trainset, **kwargs)
    testset = torchvision.datasets.ImageFolder(
        os.path.join(root_path, "val"), transform=test_transform
    )
    testloader = torch.utils.data.DataLoader(testset, **kwargs)
    return trainloader, testloader


trainloader, testloader = get_imagenette2_loaders(
    batch_size=64, shuffle=True, num_workers=2
)

labels = [
    "tench",
    "English springer",
    "cassette player",
    "chain saw",
    "church",
    "French horn",
    "garbage truck",
    "gas pump",
    "golf ball",
    "parachute",
]

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

inv_normalize = transforms.Normalize(
    mean=[-m / s for m, s in zip(means, stds)], std=[1 / s for s in stds]
)

x, y = next(iter(trainloader))
img_grid = make_grid(x[:16])
img_grid = inv_normalize(img_grid)
plt.figure(figsize=(20, 15))
plt.imshow(img_grid.permute(1, 2, 0))
plt.axis("off")

Modèle 1 : pr-trained VGG11

In [ ]:
import torch.nn as nn
import torch

model_vgg11 = torchvision.models.vgg11(pretrained=True)
for param in model_vgg11.features:
    param.requires_grad = False

model_vgg11.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=4096, out_features=10, bias=True),
)
if torch.cuda.is_available():
    model_vgg11 = model_vgg11.cuda()

Modèle 2 : Resnet

In [ ]:
model_resnet18 = torchvision.models.resnet18(pretrained=True)
for param in model_resnet18.parameters():
    param.requires_grad = False

model_resnet18.fc = nn.Linear(model_resnet18.fc.in_features, 10)

if torch.cuda.is_available():
    model_resnet18 = model_resnet18.cuda()

Entraînement, test et évaluation

In [ ]:
from tqdm import tqdm

criterion_classifier = nn.CrossEntropyLoss(reduction="mean")


def train(model, optimizer, trainloader, epochs=30):
    t = tqdm(range(epochs))
    for epoch in t:
        corrects = 0
        total = 0
        for x, y in trainloader:
            loss = 0
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            y_hat = model(x)

            loss += criterion_classifier(y_hat, y)
            _, predicted = y_hat.max(1)
            corrects += predicted.eq(y).sum().item()
            total += y.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            t.set_description(
                f"epoch: {epoch}; current accuracy: {round(corrects / total * 100, 2)}%  "
            )
    return corrects / total

In [ ]:
learning_rate = 5e-3
epochs = 1

In [ ]:
# vgg11
optimizer = torch.optim.Adam(model_vgg11.classifier.parameters(), lr=learning_rate)
train(model_vgg11, optimizer, trainloader, epochs=epochs)

# resnet18
optimizer = torch.optim.Adam(model_resnet18.fc.parameters(), lr=learning_rate)
train(model_resnet18, optimizer, trainloader, epochs=epochs)

In [ ]:
def test(model, dataloader):
    test_corrects = 0
    total = 0
    with torch.no_grad():
        for x, y in dataloader:
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            y_hat = model(x).argmax(1)
            test_corrects += y_hat.eq(y).sum().item()
            total += y.size(0)
    return test_corrects / total

In [ ]:
model_vgg11.eval()
test_acc = test(model_vgg11, testloader) * 100
print(f"Test accuracy vgg11: {test_acc:.2f} %")

model_resnet18.eval()
test_acc = test(model_resnet18, testloader) * 100
print(f"Test accuracy resnet18: {test_acc:.2f} %")

Méthode 1 : RISE

In [ ]:
import numpy as np
from skimage.transform import resize


def generate_masks(N, s, p1, image_size):
    cell_size = np.ceil(np.array(image_size) / s)
    up_size = (s + 1) * cell_size

    grid = np.random.rand(N, s, s) < p1
    grid = grid.astype("float32")

    masks = np.empty((N, *image_size))

    for i in range(N):
        # Random shifts
        x = np.random.randint(0, cell_size[0])
        y = np.random.randint(0, cell_size[1])
        # Linear interpolation
        masks[i, :, :] = resize(
            grid[i], up_size, order=1, mode="reflect", anti_aliasing=False
        )[x : x + image_size[0], y : y + image_size[1]]

    masks = masks.reshape(-1, 1, *image_size)
    masks = torch.from_numpy(masks).float()

    return masks

In [ ]:
def explain(model, N, p1, img, masks):
    img = img.unsqueeze(0).to("cpu")
    _, _, H, W = img.size()
    print(type(img), type(masks))
    stack = torch.mul(masks, img)

    p = []

    for i in tqdm(range(N)):
        with torch.no_grad():
            input = stack[i : i + 1]
            if torch.cuda.is_available():
                input = input.cuda()
            output = model(input)
            p.append(output.to("cpu"))

    p = torch.cat(p)

    CL = p.size(1)
    sal = torch.matmul(p.data.transpose(0, 1), masks.view(N, H * W))
    sal = sal.view((CL, H, W))
    sal = sal / N / p1
    return sal

In [ ]:
# Visualisation de la importance map pour la prédiction des modèles sur l'image suivante

idx = 0

img = inv_normalize(x[idx])
np_img = np.transpose(img.cpu().detach().numpy(), (1, 2, 0)) * 255
np_img = np_img.astype(np.uint8)
plt.imshow(np_img)
plt.axis("off")
input = x[idx].unsqueeze(0)
if torch.cuda.is_available():
    input = input.cuda()

In [ ]:
output = model_vgg11(input)
_, prediction = torch.topk(output, 1)
print(f"VGG11 prediction: {labels[prediction.item()]} (item number {prediction.item()})")

output = model_resnet18(input)
_, prediction = torch.topk(output, 1)
print(f"Resnet18 prediction: {labels[prediction.item()]} (item number {prediction.item()})")

In [ ]:
N = 10000  # Number of masks
s = 8  # Size of grid
p1 = 0.1  # Probability of inclusion

masks = generate_masks(N, s, p1, img.shape[-2:])

In [ ]:
saliency_maps_vgg11 = explain(model_vgg11, N, p1, img, masks)
saliency_maps_resnet18 = explain(model_resnet18, N, p1, img, masks)

In [ ]:
# vgg11
plt.imshow(np_img)

plt.imshow(saliency_maps_vgg11[4], cmap="turbo", alpha=0.5)
plt.colorbar()
plt.show()

In [ ]:
# resnet18
plt.imshow(np_img)

plt.imshow(saliency_maps_resnet18[5], cmap="turbo", alpha=0.5)
plt.colorbar()
plt.show()

Méthode 2 : Grad-CAM

In [ ]:
class HookFeatures():
    def __init__(self, module):
        self.feature_hook = module.register_forward_hook(self.feature_hook_fn)
    def feature_hook_fn(self, module, input, output):
        self.features = output.clone().detach()
        self.gradient_hook = output.register_hook(self.gradient_hook_fn)
    def gradient_hook_fn(self, grad):
        self.gradients = grad
    def close(self):
        self.feature_hook.remove()
        self.gradient_hook.remove()

In [ ]:
print(model_vgg11)
hook = HookFeatures(model_vgg11.features[19])

In [ ]:
output = model_vgg11(input)
output_idx = output.argmax()
output_max = output[0, output_idx]
output_max.backward()

gradients = hook.gradients
activations = hook.features
pooled_gradients = torch.mean(gradients, dim=[0, 2, 3]) # we take the average gradient of every chanels
for i in range(activations.shape[1]):
    activations[:, i, :, :] *= pooled_gradients[i] # we multiply every chanels of the feature map with their corresponding averaged gradients

In [ ]:
print(img.shape)

In [ ]:
import cv2

heatmap = torch.mean(activations, dim=1).squeeze()
heatmap = np.maximum(heatmap.detach().cpu(), 0)
heatmap /= torch.max(heatmap)
heatmap = cv2.resize(np.float32(heatmap), (img.shape[1], img.shape[2]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_RAINBOW) / 255
superposed_img = (heatmap) * 0.4 + np_img
plt.figure(figsize=(8,8))
plt.imshow(np.clip(superposed_img,0,1))
plt.axis('off')
plt.show()

In [ ]:
hook.close()

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets


@interact(idx=widgets.IntSlider(min=0, max=len(saliency_maps) - 1, step=1, value=0))
def visualize_saliency(idx):
    img = inv_normalize(x[idx])
    np_img = np.transpose(img.cpu().detach().numpy(), (1, 2, 0)) * 255
    np_img = np_img.astype(np.uint8)
    plt.imshow(np_img)
    # numero de la classe prédite, code a modifier car doit pas marcher
    maps_idx = y[idx].item()
    plt.imshow(saliency_maps[maps_idx], cmap="turbo", alpha=0.5)
    plt.colorbar()
    plt.show()